In [51]:
import pandas as pd
import re

In [52]:
df = pd.read_csv("C:/study/sentiment-analysis/data/raw/reddit.csv")

In [53]:
df.head()

,subreddit,body,controversiality,score
0,gameofthrones,Your submission has been automatically removed...,0,1
1,aww,"Dont squeeze her with you massive hand, you me...",0,19
2,gaming,It's pretty well known and it was a paid produ...,0,3
3,news,You know we have laws against that currently c...,0,10
4,politics,"Yes, there is a difference between gentle supp...",0,1


In [54]:
df.describe()

,controversiality,score
count,1000000.000000,1000000.000000
mean,0.029583,11.510103
std,0.169434,149.671560
min,0.000000,-889.000000
25%,0.000000,1.000000
50%,0.000000,2.000000
75%,0.000000,4.000000
max,1.000000,35619.000000


In [55]:
def is_junk(text): 
    if text is None:
        return True
    text = text.strip()
    if len(text)  < 3:
        return True
    if re.fullmatch(r"[^/w/s]+", text):
        return True
    if re.fullmatch(r"(http\S+|www\S+)", text):
        return True
    return False


In [56]:
df["is_junk"] = df["body"].apply(is_junk)

In [57]:
df = df[df["is_junk"] == False]

In [58]:
df.shape

(931210, 5)

In [59]:
df.subreddit.value_counts()

subreddit
asoiaf                 24629
AmItheAsshole          24610
relationship_advice    24540
news                   24520
worldnews              24492
unpopularopinion       24476
politics               24408
gameofthrones          24398
movies                 24315
videos                 24261
todayilearned          24224
leagueoflegends        24216
marvelstudios          24199
apexlegends            24141
nfl                    24108
Showerthoughts         24012
trashy                 24009
soccer                 23987
ChapoTrapHouse         23984
SquaredCircle          23955
MortalKombat           23927
pics                   23826
gaming                 23796
AskReddit              23751
The_Donald             23750
freefolk               23739
FortNiteBR             23641
nba                    23511
hockey                 23508
funny                  23476
wallstreetbets         23370
RoastMe                23338
aww                    23222
Market76               23120
dank

In [60]:
high_score = df[df["score"] >= 10]     
low_score  = df[df["score"] <= -5]      
mid_score  = df[(df["score"] > -2) & (df["score"] < 5)]  
contro = df[df["controversiality"] == 1]

In [61]:
high_s = high_score.sample(8000, random_state=42)
low_s  = low_score.sample(7000, random_state=42)
mid_s  = mid_score.sample(6000, random_state=42)
contro_s = contro.sample(4000, random_state=42)

In [62]:
sampled = pd.concat([
    high_score[high_score["controversiality"] == 0].sample(8000, random_state=42),
    low_score[low_score["controversiality"] == 0].sample(8000, random_state=42),
    mid_score[mid_score["controversiality"] == 0].sample(5000, random_state=42),
    contro.sample(4000, random_state=42),
])

In [63]:
final_df = (
    pd.concat([high_s, low_s, mid_s, contro_s])
      .drop_duplicates()   
      .sample(frac=1, random_state=42)
)

In [64]:
final_df.shape

(24760, 5)

In [65]:
final_df["score"].describe()

count    24760.000000
mean        21.809612
std        240.490025
min       -561.000000
25%         -5.000000
50%          1.000000
75%         13.000000
max      17474.000000
Name: score, dtype: float64

In [66]:
final_df["subreddit"].value_counts().head()


subreddit
news              1252
worldnews         1051
soccer            1004
AmItheAsshole      998
ChapoTrapHouse     922
Name: count, dtype: int64

In [67]:
final_df["controversiality"].value_counts(normalize=True)


controversiality
0    0.804564
1    0.195436
Name: proportion, dtype: float64

In [68]:
final_df.to_csv("processed-reddit-subreddits.csv")